In [2]:
"""
The Full Whiskas Model Python Formulation for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""

# Import PuLP modeler functions
!conda install -c conda-forge pulp --yes
from pulp import *


Solving environment: done

## Package Plan ##

  environment location: /home/misp/anaconda3

  added / updated specs:
    - pulp


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.1 MB

The following packages will be UPDATED:

  openssl                                 1.1.1c-h516909a_0 --> 1.1.1d-h516909a_0



openssl-1.1.1d       | 2.1 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Największy prąd",LpMaximize)

# The 2 variables Beef and Chicken are created with a lower limit of zero
Iwej=LpVariable("I wej",0)
Ir1=LpVariable("I R1",0)
Ir2=LpVariable("I R2",0)
Ir4=LpVariable("I R4",0)
Ir5=LpVariable("I R5",0)

# The objective function is added to 'prob' first
prob += Iwej, "Total Cost of Ingredients per can"

# The five constraints are entered
prob += Iwej <= 4, "Maks I R3"
prob += Ir1 <= 2, "Maks I R1"
prob += Ir2 <= 3, "Maks I R2"
prob += Ir4 <= 2, "Maks I R4"
prob += Ir5 <= 2, "Maks I R5"
prob += Iwej * (6/14) == Ir1, "I R1"
prob += Iwej * (8/14) == Ir2, "I R2"
prob += Iwej * (10/18) == Ir4, "I R4"
prob += Iwej * (8/18) == Ir5, "I R5"


# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print ("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print (v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print ("Maks Iwej = ", value(prob.objective))


Status: Optimal
I_R1 = 1.5428571
I_R2 = 2.0571429
I_R4 = 2.0
I_R5 = 1.6
I_wej = 3.6
Maks Iwej =  3.6


In [7]:
?? interact

Object ` interact` not found.


In [5]:
rom __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': 'initial'}


cena_jablek_slider = FloatSlider(min=0,max = 1, value= 0.3, description="cena_jablek_slider", style= style )
cena_miodu_slider = FloatSlider(min=0,max = 1,value= 0.2, description="cena_miodu_slider", style= style )
zysk_z_dzemu_slider = FloatSlider(min=0,max = 50, value= 16, description="zysk_z_dzemu_slider", style= style )
zysk_z_musu_slider = FloatSlider(min=0,max = 50, value= 11, description="zysk_z_musu_slider", style= style )
zuzycie_jablek_dzem_slider = FloatSlider(min=0,max = 20,value= 8, description="zuzycie_jablek_dzem_slider", style= style )
zuzycie_jablek_mus_slider = FloatSlider(min=0,max = 20,value= 2, description="zuzycie_jablek_mus_slider", style= style )
zuzycie_miodu_dzem_slider = FloatSlider(min=0,max = 20,value= 5, description="zuzycie_miodu_dzem_slider", style= style )
zuzycie_miodu_mus_slider = FloatSlider(min=0,max = 20,value= 5, description="zuzycie_miodu_mus_slider", style= style )
maksimum_jablek_slider = IntSlider(min=0,max = 200,value= 40, description="maksimum_jablek_slider", style= style )
maksimum_miodu_slider = FloatSlider(min=0,max = 200,value= 25, description="maksimum_miodu_slider", style= style )


def hurt_produkcja(cena_jablek = 0.3,
    cena_miodu = 0.2,
    zysk_z_dzemu = 16,
    zysk_z_musu = 11,
    zuzycie_jablek_dzem = 8,
    zuzycie_jablek_mus = 2,
    zuzycie_miodu_dzem = 5,
    zuzycie_miodu_mus = 5,
    maksimum_jablek = 40,
    maksimum_miodu = 25):
    
    prob = LpProblem("Najlepsza produkcja HURT",LpMaximize)

    # 2 zmienne
    dzem=LpVariable("Dzem",0) # juz tutaj podajemy ograniczenie ze sa nieujemne
    mus=LpVariable("Mus",0)
    # pomocnicze zmienne
    zuzycie_jablek =LpVariable("Jablka",0)
    zuzycie_miodu =LpVariable("Miod",0)

    prob += zysk_z_dzemu * dzem + zysk_z_musu * mus - cena_jablek * zuzycie_jablek - cena_miodu * zuzycie_miodu
    prob += zuzycie_jablek_dzem * dzem + zuzycie_jablek_mus * mus == zuzycie_jablek, "Zuzycie jablek"
    prob += zuzycie_miodu_dzem * dzem + zuzycie_miodu_mus * mus == zuzycie_miodu, "Zuzycie miodu"
    prob += zuzycie_jablek <= maksimum_jablek, "Maksimum jablek"
    prob += zuzycie_miodu <= maksimum_miodu, "Maksimum miodu"

    
    prob.solve()
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[prob.status])
    # Each of the variables is printed with it's resolved optimum value
    for v in prob.variables():
        print(v.name, "=", v.varValue)
    print("Zysk wyniesie {} zł  ".format( value(prob.objective)))
    
    shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
    print("*"*10+"Shadow Prices"+"*"*10)
    print(pd.DataFrame(shadows))
    
    
interact(hurt_produkcja,cena_jablek = cena_jablek_slider,
    cena_miodu =    cena_miodu_slider,
    zysk_z_dzemu =    zysk_z_dzemu_slider,
    zysk_z_musu =zysk_z_musu_slider,
    zuzycie_jablek_dzem =zuzycie_jablek_dzem_slider,
    zuzycie_jablek_mus =zuzycie_jablek_mus_slider,
    zuzycie_miodu_dzem =zuzycie_miodu_dzem_slider,
    zuzycie_miodu_mus =zuzycie_miodu_mus_slider,
    maksimum_jablek =maksimum_jablek_slider,
    maksimum_miodu =maksimum_miodu_slider)


SyntaxError: invalid syntax (<ipython-input-5-42e73d7f0a9c>, line 1)

In [5]:
from ipywidgets import IntSlider

style = {'description_width': 'initial'}
IntSlider(description='A too long description', style=style)

IntSlider(value=0, description='A too long description', style=SliderStyle(description_width='initial'))

In [6]:

x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget)

NameError: name 'FloatSlider' is not defined